In [ ]:
# !pip uninstall -y tensorflow
# !pip install tensorflow==2.2.0 tensorflow_gcs_config==2.2.0
# !pip install cloud-tpu-client

In [ ]:
# import tensorflow as tf
# print(tf.__version__)

# from cloud_tpu_client import Client
# Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')

In [ ]:
import numpy as np
import pandas as pd
import os, sys, cv2
from kaggle_datasets import KaggleDatasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping,LearningRateScheduler

In [ ]:
# data.csv excluded id_freq>150
df= pd.read_csv('../input/dataframe-startnotebook/data.csv')
df_triplet= pd.read_csv('../input/dataframe-startnotebook/tripletData.csv')
Encoder=LabelEncoder()
df['id_label']=Encoder.fit_transform(df.individual_id)
np.save('classes.npy', Encoder.classes_)
# enc.classes_ = np.load('classes.npy', allow_pickle=True)
# enc.inverse_transform([y1, y2])
df.head()

In [ ]:
print(len(df_triplet))
df_triplet.head()

In [ ]:
df_triplet2= df_triplet.dropna()
df_triplet= df_triplet[['anchor', 'positive', 'negative0']]
newDict= {'anchor':[], 'positive':[], 'negative0':[]}
for i, row in df_triplet2.iterrows():
    newDict['anchor']+= [row.anchor, row.anchor]
    newDict['positive']+= [row.positive, row.positive]
    newDict['negative0']+= [row.negative1, row.negative2]
df_triplet2= pd.DataFrame(newDict)
df_triplet= pd.concat([df_triplet, df_triplet2])
df_triplet = df_triplet.sample(frac=1).reset_index(drop=True)
df_triplet.info()

In [ ]:
img_size = 456
n_epochs = 5
lr= 0.0001
val_split= 0.1
seed= 20
batch_size=16

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        
        # set half precision policy
        # Mixed precision is the use of both 16-bit and 32-bit floating-point types
        # in a model during training to make it run faster and use less memory.
        mixed_precision.set_policy('mixed_bfloat16' if TPU else 'float32')
        print(f'Compute dtype: {mixed_precision.global_policy().compute_dtype}')
        print(f'Variable dtype: {mixed_precision.global_policy().variable_dtype}')
        
        # enable XLA optmizations
        tf.config.optimizer.set_jit(True)
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

In [ ]:
def readImg(target_size=(512, 512)):
    def readOnly(path):
        file_bytes = tf.io.read_file(path)
        img = tf.image.decode_jpeg(file_bytes, channels=3)
        img= tf.cast(img, tf.float32)/255.0
        return tf.image.resize(img, target_size)
    def read3(row):
        path1, path2, path3= row['anchor'], row['positive'], row['negative0']
        return readOnly(path1), readOnly(path2), readOnly(path3)
    return read3


def build_dataset(df, bsize=20,
                  decode_fn=None, repeat=True, buffer=200):
    if decode_fn is None:
        decode_fn = readImg()
        
    AUTO = tf.data.experimental.AUTOTUNE
    dset = tf.data.Dataset.from_tensor_slices(dict(df))
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(buffer) if buffer else dset
    dset = dset.batch(bsize).prefetch(AUTO) # overlaps data preprocessing and model execution while training
    return dset

In [ ]:
TPU=True
DATASET_NAME = "happy-whale-and-dolphin"
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * batch_size
print('batch size', batch_size)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path(DATASET_NAME)
GCS_DS_PATH #   ='gs://kds-129aef496145c7329bf35723a6d8c89a4a331d719ecacd84746ce6b8'

In [ ]:
df_triplet = df_triplet.apply(lambda x: GCS_DS_PATH+ '/train_images/' + x)
train_paths, val_paths = train_test_split(df_triplet, test_size=val_split, random_state=seed)
print(len(train_paths), len(val_paths))
train_paths.head()

In [ ]:
decoder = readImg(target_size=(img_size, img_size))

# Build the tensorflow datasets
dtrain = build_dataset(
    train_paths, bsize=batch_size, decode_fn=decoder)

dvalid = build_dataset(
    val_paths, bsize=batch_size*2, 
    repeat=False, buffer=False, decode_fn=decoder)

In [ ]:
data0,data1= dtrain.take(2)

images=[]
for i in range(3):
    images+=data0[i][:4].numpy().tolist()

In [ ]:
#ANCHOR
#POSITIVE 
#NEGATIVE
fig, axes = plt.subplots(3, 4, figsize=(20,10))
axes = axes.flatten()
for img, ax in zip(images, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
class eluDistance(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

In [ ]:
def buildModel():
    anchor_input = layers.Input(name="anchor", shape=(img_size, img_size, 3))
    positive_input = layers.Input(name="positive", shape=(img_size, img_size, 3))
    negative_input = layers.Input(name="negative", shape=(img_size, img_size, 3))
    
    base= tf.keras.applications.ResNet50V2(input_shape=(img_size, img_size, 3),
                                           include_top=False, pooling='avg')
    for layer in base.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True
    
    dropout = layers.Dropout(0.25, name='dropout')
    reduce = layers.Dense(512, activation='linear', name='reduce')
    
    distances = eluDistance()(
        reduce(dropout(base(anchor_input))),
        reduce(dropout(base(positive_input))),
        reduce(dropout(base(negative_input))),
    )
    
    return  tf.keras.Model(inputs=[anchor_input, positive_input, negative_input], outputs=distances)

In [ ]:
#https://www.kaggle.com/xhlulu/shopee-siamese-resnet-50-with-triplet-loss-on-tpu

class SiameseModel(tf.keras.Model):
    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        
    def call(self, inputs):
        return self.siamese_network(inputs)
    
    def _compute_loss(self, data):
        ap_distance, an_distance= self.siamese_network(data)
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}
    
    def test_step(self, data):
        loss = self._compute_loss(data)
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}
    @property
    def metrics(self):
        return [self.loss_tracker]

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 5.0
    lrate = initial_lrate* math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

LR_START = 1e-5
LR_MAX = 0.0002
LR_RAMPUP_EPOCHS = 2
LR_SUSTAIN_EPOCHS = 1
LR_STEP_DECAY = 0.7

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//2)
    return lr

lrate = LearningRateScheduler(lrfn)

In [ ]:
name= 'SiameseResNet50V2.h5'
ckp = ModelCheckpoint(name,monitor = 'val_loss',
                      verbose = 1, save_best_only = True, mode = 'min')
        
es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 5, mode = 'min', 
                    restore_best_weights = True, verbose = 1)

In [ ]:
with strategy.scope():
    model=buildModel()
    siamese_model = SiameseModel(model)
    siamese_model.compile(optimizer=Adam(lr))
model.summary()

In [ ]:
steps_per_epoch = ((train_paths.shape[0] // batch_size)//100)*100-100
steps_per_epoch

In [ ]:
history = siamese_model.fit(dtrain,                      
                    validation_data=dvalid,                                       
                    epochs=n_epochs,
                    callbacks=[es,ckp,lrate],
                    steps_per_epoch=steps_per_epoch,
                    verbose=1)

In [ ]:
plt.figure(figsize = (12, 6))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot( history.history["loss"], label = "Training Loss", marker='o')
plt.plot( history.history["val_loss"], label = "Validation Loss", marker='+')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
_= siamese_model(data0)
siamese_model.load_weights(name)

In [ ]:
with strategy.scope():
    encoder = tf.keras.Sequential([
        siamese_model.siamese_network.get_layer('resnet50v2'),
        siamese_model.siamese_network.get_layer('dropout'),
        siamese_model.siamese_network.get_layer('reduce'),
    ])

    encoder.save('encoder.h5')